# RFM Analysis code

This iPython notebook explains how to perform RFM analysis from customer purchase history data.

The sample orders file is Sample - Superstore dataset from Tableau Software.

if you have suggetions or want to contribute, please raise an issue or send a merge request.

 ------
 
  First step is to include all the library that we would be using during the analysis.  

  I am also suppressing unnecessary warning messages for a cleaner output.

In [1]:
%matplotlib inline
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

-----
adding some style to have a better table display and network display

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

I reading the order file, containing all the past purchase information as well as parsing the order_date column so that python is able to recognize it.  I am also passing some more arguments for a smoother analysis process, they are optional.

In [3]:
orders = pd.read_csv('./data.csv', encoding='UTF-8',
                     parse_dates=['order_date'],
                     infer_datetime_format=True,
                     index_col=None)
orders.head()

,order_date,order_id,customer,grand_total
0,2011-09-07,CA-2011-100006,Dennis Kane,378
1,2011-07-08,CA-2011-100090,Ed Braxton,699
2,2011-03-14,CA-2011-100293,Neil Franz?sisch,91
3,2011-01-29,CA-2011-100328,Jasper Cacioppo,4
4,2011-04-08,CA-2011-100363,Jim Mitchum,21


In [4]:
start = orders.order_date.min()
print('start date', start)
end = orders.order_date.max()
print('end date', end)

start date 2011-01-04 00:00:00
end date 2014-12-31 00:00:00


In [5]:
print('total duration', end - start)

total duration 1457 days 00:00:00


So based on the maximum date, creating a refernce point which would be used in calculating recency in the next steps.

In [6]:
import datetime as dt
NOW = dt.datetime(2014,12,31)
NOW

datetime.datetime(2014, 12, 31, 0, 0)

### Create the RFM Table

In [7]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency calculation
                                        'order_id': lambda x: len(x),      # Frequency calculation
                                        'grand_total': lambda x: x.sum()}) # Monetary Value calculation

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


Manually verifying by filtering data by one customer and checking his data against excel

In [8]:
test_df = orders[orders.customer == 'Aaron Hawkins']
test_df.reset_index(drop=True)
print('frequency value should be', len(test_df))
print('recency   value should be', (NOW - test_df.order_date.max()).days)
print('monetory  value should be', test_df.grand_total.sum())

frequency value should be 7
recency   value should be 12
monetory  value should be 1744


### So the table is correct, so now moving forward to creating dynamic limit for each segment

In [9]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


now we have the value for each quantile, we would convert it to dictionary for easier and scalable data processing

In [10]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

### making the copy of the data to create customer segments

In [11]:
rfmSegmentation = rfmTable.copy()

Now creating function which will actually process each customer and tag them with appropriate segment.

We want Frequency and Monetory value to be high but recency has to be low because high recency customer infers that he/she had purchased long time back whereas the low recency value infers they had visited the store recently.

In [12]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [13]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

creating new column for RFM Analysis score

In [14]:
rfmSegmentation['RFMScore'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

### Let's look at our final output of the analysis

In [15]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMScore
customer,,,,,,,
Aaron Bergman,415,3,887,4,4,4,444
Aaron Hawkins,12,7,1744,1,2,3,123
Aaron Smayling,88,7,3050,3,2,2,322
Adam Bellavance,54,8,7756,2,2,1,221
Adam Hart,34,10,3249,2,1,2,212


In [16]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv')

### So next step would be to be most effective lets list 5 top customer by monetory value in each of segment so that we can target them with the customized marketing strategy

In [17]:
# remove the comment below if you want to display customer name for each segment
# segment_list = rfmSegmentation.RFMScore.unique()
# for segment in segment_list:
#     print(rfmSegmentation[rfmSegmentation['RFMScore']==segment].sort_values('monetary_value', ascending=False).head(5))


### for display purpose I am only going to display 5 best customer

In [18]:
rfmSegmentation[rfmSegmentation['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMScore
customer,,,,,,,
Sanjit Engle,9,11,12210,1,1,1,111
John Lee,21,11,9801,1,1,1,111
Pete Kriz,9,12,8647,1,1,1,111
Harry Marie,2,10,8237,1,1,1,111
Lena Creighton,16,12,7661,1,1,1,111
